In [7]:
from os.path import join

from random import sample, seed

from pandas import read_csv, DataFrame

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score

In [12]:
data_dir = "/home/benoit/data/dreem_challenge/"
labels_path = "/home/benoit/data/dreem_challenge/y_train.csv"

labels = read_csv(labels_path)

In [13]:
from tools.processing import load_features, merge_features

features_1 = load_features(join(data_dir, "computed", 'base_1_train.csv'))
features_2 = load_features(join(data_dir, "computed", 'base_2_train.csv'))

f_train = merge_features([features_1, features_2])
y = labels
f_train = f_train.drop(['abs_index'], axis=1)
training_features_list = f_train.keys()

Index(['eeg_0_log_ampl', 'eeg_0_mean', 'eeg_0_log_energy', 'eeg_0_kurtosis',
       'eeg_1_log_ampl', 'eeg_1_mean', 'eeg_1_log_energy', 'eeg_1_kurtosis',
       'eeg_2_log_ampl', 'eeg_2_mean', 'eeg_2_log_energy', 'eeg_2_kurtosis',
       'eeg_3_log_ampl', 'eeg_3_mean', 'eeg_3_log_energy', 'eeg_3_kurtosis',
       'eeg_4_log_ampl', 'eeg_4_mean', 'eeg_4_log_energy', 'eeg_4_kurtosis',
       'eeg_5_log_ampl', 'eeg_5_mean', 'eeg_5_log_energy', 'eeg_5_kurtosis',
       'eeg_6_log_ampl', 'eeg_6_mean', 'eeg_6_log_energy', 'eeg_6_kurtosis',
       'eeg_0_skewness', 'eeg_0_log_variance', 'eeg_0_zero_crossings',
       'eeg_1_skewness', 'eeg_1_log_variance', 'eeg_1_zero_crossings',
       'eeg_2_skewness', 'eeg_2_log_variance', 'eeg_2_zero_crossings',
       'eeg_3_skewness', 'eeg_3_log_variance', 'eeg_3_zero_crossings',
       'eeg_4_skewness', 'eeg_4_log_variance', 'eeg_4_zero_crossings',
       'eeg_5_skewness', 'eeg_5_log_variance', 'eeg_5_zero_crossings',
       'eeg_6_skewness', 'eeg_6_log

In [7]:
X_train, X_test, y_train, y_test = train_test_split(f_train, y, test_size = 0.1)

# Features preprocessing (noramlization etc...)

# ...

# ...

#

In [1]:
# Feature Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# slct = SelectKBest(f_classif, k=10)

# X_train = slct.fit_transform(X_train, y_train)
# X_test = slct.transform(X_test)


In [4]:
# Model initialization

clf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None)
#clf = KNeighborsClassifier(n_neighbors = 15)
#clf= SVC(kernel = 'linear', C = 1)
#clf = GaussianNB()

In [5]:
# Hyperparameter tuning (k-fold etc...)

# ...

# ...

#

In [ ]:
# Model training

clf.fit(X_train, y_train)

predictions = clf.predict(X_train)
print(f"Training score - {f1_score(predictions, y_train, average='weighted')}")

predictions = clf.predict(X_test)
print(f"Testing score - {f1_score(predictions, y_test, average='weighted')}")

In [14]:
# Loading evaluation data

f_1_e = load_features(join(data_dir, "computed", 'base_1_eval.csv'))
f_2_e = load_features(join(data_dir, "computed", 'base_2_eval.csv'))

f_eval = merge_features([f_1_e, f_2_e])

f_eval = f_eval.drop(['abs_index'], axis=1)

assert f_train.keys() == f_eval.keys()

# Features preprocessing & Feature selection (both must be applied as for training features)

# ...

# ...

#

In [ ]:
# Making prediction on eval data

eval_pred = clf.predict(f_eval)

sub = read_csv(join(data_dir, 'sample_submission.csv'))
sub['sleep_stage'] = eval_pred

sub.to_csv("output/submission.csv", index=False)